## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,2021-04-25 16:11:40,-53.1500,-70.9167,57.20,47,40,21.85,scattered clouds
1,1,Georgetown,MY,2021-04-25 16:11:43,5.4112,100.3354,82.40,83,20,4.61,few clouds
2,2,Lompoc,US,2021-04-25 16:08:20,34.6391,-120.4579,53.60,71,100,3.44,overcast clouds
3,3,Thompson,CA,2021-04-25 16:11:39,55.7435,-97.8558,28.40,54,75,8.05,broken clouds
4,4,Lavrentiya,RU,2021-04-25 16:11:46,65.5833,-171.0000,20.23,87,100,3.71,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                   (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Georgetown,MY,2021-04-25 16:11:43,5.4112,100.3354,82.40,83,20,4.61,few clouds
6,6,Mount Isa,AU,2021-04-25 16:14:45,-20.7333,139.5000,71.60,43,39,5.75,light rain
7,7,Kirakira,SB,2021-04-25 16:12:14,-10.4544,161.9205,78.80,84,66,3.38,broken clouds
9,9,Victoria,HK,2021-04-25 16:08:40,22.2855,114.1577,75.00,93,100,1.01,overcast clouds
10,10,East London,ZA,2021-04-25 16:12:30,-33.0153,27.9116,69.80,78,0,8.05,clear sky
14,14,Arraial Do Cabo,BR,2021-04-25 16:11:58,-22.9661,-42.0278,82.40,61,0,9.22,clear sky
17,17,Bandarbeyla,SO,2021-04-25 16:14:47,9.4942,50.8122,83.91,74,43,11.50,scattered clouds
18,18,Faanui,PF,2021-04-25 16:07:54,-16.4833,-151.7500,80.53,66,7,3.51,clear sky
19,19,Chuy,UY,2021-04-25 16:13:12,-33.6971,-53.4616,67.75,54,9,12.06,clear sky
20,20,Hambantota,LK,2021-04-25 16:13:54,6.1241,81.1185,80.60,100,75,1.14,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                275
City                   275
Country                272
Date                   275
Lat                    275
Lng                    275
Max Temp               275
Humidity               275
Cloudiness             275
Wind Speed             275
Current Description    275
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                272
City                   272
Country                272
Date                   272
Lat                    272
Lng                    272
Max Temp               272
Humidity               272
Cloudiness             272
Wind Speed             272
Current Description    272
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,82.40,few clouds,5.4112,100.3354,
6,Mount Isa,AU,71.60,light rain,-20.7333,139.5000,
7,Kirakira,SB,78.80,broken clouds,-10.4544,161.9205,
9,Victoria,HK,75.00,overcast clouds,22.2855,114.1577,
10,East London,ZA,69.80,clear sky,-33.0153,27.9116,
14,Arraial Do Cabo,BR,82.40,clear sky,-22.9661,-42.0278,
17,Bandarbeyla,SO,83.91,scattered clouds,9.4942,50.8122,
18,Faanui,PF,80.53,clear sky,-16.4833,-151.7500,
19,Chuy,UY,67.75,clear sky,-33.6971,-53.4616,
20,Hambantota,LK,80.60,broken clouds,6.1241,81.1185,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,82.4,few clouds,5.4112,100.3354,Cititel Penang
6,Mount Isa,AU,71.6,light rain,-20.7333,139.5000,ibis Styles Mt Isa Verona
7,Kirakira,SB,78.8,broken clouds,-10.4544,161.9205,SanBiz Lodge
9,Victoria,HK,75.0,overcast clouds,22.2855,114.1577,Mini Hotel Central
10,East London,ZA,69.8,clear sky,-33.0153,27.9116,Tu Casa


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()


In [11]:
# 8a. Create the output File (CSV)
WeatherPy_vacation = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(WeatherPy_vacation, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} {Max Temp}°F </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.3)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))